# Common utitities for all production alphas

In [1]:
import NBInclude: @nbinclude
@nbinclude("../TrainingAlphas/Alpha.ipynb");

In [2]:
if !@isdefined source
    source = "Alpha"
end;

In [3]:
function get_recommendee_username()
    dir = "../../data/recommendations"
    open("$dir/recommendee_username.txt") do f
        read(f, String)
    end
end;

In [4]:
set_logging_outdir(
    "../../data/recommendations/$(get_recommendee_username())/alphas/$source",
);

## Reading and writing data

In [5]:
function get_recommendee_split(implicit::Bool)
    if implicit
        df = cat(
            get_raw_recommendee_split("explicit"),
            get_raw_recommendee_split("implicit"),
        )
        df.rating .= 1
    else
        df = get_raw_recommendee_split("explicit")
    end
    df
end

function get_raw_recommendee_split(split)
    @assert split in ["explicit", "implicit"]
    username = get_recommendee_username()
    dir = "../../data/recommendations"
    file = "$dir/$username/splits.jld2"
    JLD2.load(file, split)
end;

In [6]:
function recommendee_alpha_basepath()
    dir = "../../data/recommendations"
    username = get_recommendee_username()
    mkpath("$dir/$username/alphas")
end;

In [7]:
function write_recommendee_alpha(ratings, outdir)
    @assert length(ratings) == num_items()
    items = 1:num_items()
    users = fill(1, length(items))
    alpha = RatingsDataset(users, items, ratings)
    outdir = mkpath("$(recommendee_alpha_basepath())/$outdir")
    JLD2.save("$outdir/alpha.jld2", Dict("alpha" => alpha))
end;

In [8]:
function read_recommendee_alpha(alpha, split)
    @assert split in ["explicit", "implicit", "all"]
    df = JLD2.load("$(recommendee_alpha_basepath())/$alpha/alpha.jld2")["alpha"]
    if split == "all"
        return df
    else
        recommendee_split = split == "implicit" ? true : false
        mask = get_recommendee_split(recommendee_split).item
        return RatingsDataset(df.user[mask], df.item[mask], df.rating[mask])
    end
end;

In [9]:
function read_recommendee_alpha(alphas::Vector, implicit)
    _, β = regress(alphas, implicit)
    ratings = zeros(Float32, num_items())
    for i = 1:length(alphas)
        ratings += β[i] .* read_recommendee_alpha(alphas[i], "all").rating
    end
    if implicit
        @assert length(alphas) + 1 == length(β)
        ratings .+= β[end] * 1 / num_items()
    end

    df = get_recommendee_split(implicit)
    RatingsDataset(df.user, df.item, ratings[df.item])
end;

## Dependecy management

In [10]:
function reset_recommenndee_alphas()
    rm(recommendee_alpha_basepath(), recursive = true)
end;

In [11]:
function should_compute(alpha)
    dir = recommendee_alpha_basepath()
    # see if we've already computed this alpha
    if isfile("$dir/alpha/alpha.jld2")
        return false
    end

    # see if we have the dependencies necessary to compute the alpha
    residual_alphas = missing
    for key in ["residual_alphas", "training_residuals"]
        if key in keys(read_params(alpha))
            residual_alphas = read_params(alpha)[key]
        end
    end
    if ismissing(residual_alphas)
        @info "Could not find residual alphas for $alpha"
    end
    for α in residual_alphas
        if !isfile("$dir/$α/alpha.jld2")
            return false
        end
    end

    # we're good to go!
    true
end;